In [1]:
import pandas as pd
import sqlalchemy as sa
import numpy as np
import pyodbc 
from sqlalchemy import create_engine

# 1. load this data in sql and in pandas with a relation in sql

In [2]:
superstore=pd.read_excel('Superstore_USA.xlsx')

In [3]:
superstore.head(2)

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.56,4,13.01,88522


In [4]:
ret=pd.read_excel('Superstore_USA.xlsx',sheet_name="Returns")

In [5]:
ret.head(2)

,Order ID,Status
0,65,Returned
1,612,Returned


In [6]:
users=pd.read_excel('Superstore_USA.xlsx',sheet_name="Users")

In [7]:
users.head(2)

,Region,Manager
0,Central,Chris
1,East,Erin


# SQL database connection & Insert data in to SQL table

In [8]:
engine = sa.create_engine('mssql+pyodbc://(LocalDB)\MSSQLLocalDB/test?driver=SQL Server Native Client 11.0?Trusted_Connection=yes')

In [9]:
superstore.to_sql('SUPERSTORE_ORDER',engine,if_exists='replace',index=False)

In [10]:
ret.to_sql('SUPERSTORE_Returns',engine,if_exists='replace',index=False)

In [11]:
users.to_sql('SUPERSTORE_Users',engine,if_exists='replace',index=False)

# 3.Hom many returns received with product id

In [12]:
merge_superstrret=pd.merge(superstore,ret,on='Order ID',how='right')

In [13]:
merge_superstrret.head(2)

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID,Status
0,1359.0,Low,0.05,5.85,2.27,21.0,Tony Wilkins Winters,Regular Air,Small Business,Office Supplies,...,New York,New York City,10012.0,2011-04-20,2011-04-24,-6.82,9.0,54.79,9895,Returned
1,1950.0,Medium,0.01,4.91,0.50,117.0,Linda Weiss,Regular Air,Home Office,Office Supplies,...,Washington,Seattle,98103.0,2010-04-04,2010-04-06,112.06,47.0,228.46,13959,Returned


In [14]:
Product_Returncnt=pd.DataFrame(merge_superstrret.groupby(['Product Category','Product Sub-Category'])['Status'].count())

In [15]:
Product_Returncnt.rename(columns={'Status':'Status Count'})

Status Count
Product Category Product Sub-Category                        
Furniture        Bookcases                                  1
                 Chairs & Chairmats                         6
                 Office Furnishings                         7
                 Tables                                     5
Office Supplies  Appliances                                 1
                 Binders and Binder Accessories            12
                 Envelopes                                  5
                 Labels                                     3
                 Paper                                     16
                 Pens & Art Supplies                       11
                 Rubber Bands                               2
                 Scissors, Rulers and Trimmers              2
                 Storage & Organization                     6
Technology       Computer Peripherals                       9
                 Office Machines                            3
                 Telephones and Communication               9

# 4 . try  to join order and return data both in sql and pandas 

In [16]:
merge_superstrret=pd.merge(superstore,ret,on='Order ID',how='left')

In [17]:
merge_superstrret.head(3)

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID,Status
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Illinois,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.90,88525,NaN
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.56,4,13.01,88522,NaN
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,Washington,Anacortes,98221,2011-07-27,2011-07-28,-47.64,7,49.92,88523,NaN


# 5 . Try to find out how many unique customer that we have

In [18]:
'Number of unique customers = {}'.format(superstore['Customer ID'].nunique())

'Number of unique customers = 2703'

# 6 . try to find out in how many regions we are selling a product and who is a manager for a respective region 

In [19]:
merge_superretuser= pd.merge(merge_superstrret,users,on='Region',how='left')

In [20]:
merge_superretuser.head(2)

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID,Status,Manager
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.90,88525,NaN,Chris
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,Anacortes,98221,2010-07-07,2010-07-08,4.56,4,13.01,88522,NaN,William


In [21]:
'We are selling product in {} regions'.format(merge_superretuser['Region'].nunique())

'We are selling product in 4 regions'

In [22]:
merge_superretuser.groupby(['Region','Manager'])['Region'].nunique().reset_index(name='count')

,Region,Manager,count
0,Central,Chris,1
1,East,Erin,1
2,South,Sam,1
3,West,William,1


In [27]:
pd.DataFrame([merge_superretuser.groupby('Region')['Manager'].unique()])

Region,Central,East,South,West
Manager,[Chris],[Erin],[Sam],[William]


# 7. find out how many different differnet shipement mode that we have and what is a percentage usablity of all the shipment mode with respect to dataset

In [59]:
'The number of shipmet modes we have is {}'.format(superstore['Ship Mode'].nunique())

'The number of shipmet modes we have is 3'

In [62]:
Shipmode_cnt=pd.DataFrame(superstore['Ship Mode'].value_counts())

In [84]:
Shipmode_cnt['%use']=(Shipmode_cnt['Ship Mode']*100)/len('superstore')

In [85]:
Shipmode_cnt

,Ship Mode,Usablity %,%use
Regular Air,7036,75.0,70360.0
Delivery Truck,1283,14.0,12830.0
Express Air,1107,12.0,11070.0


# 8 . Create a new coulmn and try to find our a diffrence between order date and shipment date

In [101]:
superstore['Shipment time']=(superstore['Ship Date']-superstore['Order Date'])

In [98]:
superstore.head()

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID,Shipment time
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,Illinois,Addison,60101,2012-05-28,2012-05-30,1.3200,2,5.90,88525,2 days
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.5600,4,13.01,88522,1 days
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,Washington,Anacortes,98221,2011-07-27,2011-07-28,-47.6400,7,49.92,88523,1 days
3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,...,Washington,Anacortes,98221,2011-07-27,2011-07-28,-30.5100,7,41.64,88523,1 days
4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,...,Washington,Anacortes,98221,2011-07-27,2011-07-27,998.2023,8,1446.67,88523,0 days


# 9 . base on question number 8 find out for which order id we have shipment duration more than 10 days 

In [121]:
((((superstore[superstore['Shipment time']>10])[['Order ID','Shipment time']])).sort_values(by=['Shipment time'],ascending=False)).reset_index(drop=True)

,Order ID,Shipment time
0,86177,92
1,87215,84
2,19841,31
3,87300,31
4,19556,28
5,87957,28
6,87300,27
7,19841,27
8,91294,24
9,87572,22


# 10 . Try to find out a list of a returned order which sihpment duration was more then 15 days and find out that region manager as well

In [128]:
merge_superretuser['Shipment time']=(merge_superretuser['Ship Date']-merge_superretuser['Order Date']).dt.days

In [129]:
merge_superretuser.head(1)

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID,Status,Manager,Shipment time
0,18606,Not Specified,0.01,2.88,0.5,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,...,60101,2012-05-28,2012-05-30,1.32,2,5.9,88525,NaN,Chris,2


In [137]:
merge_superretuser[merge_superretuser['Shipment time']>15][['Shipment time','Order ID','Region','Manager']].sort_values(by=['Shipment time'],ascending=False).reset_index(drop=True)

,Shipment time,Order ID,Region,Manager
0,92,86177,West,William
1,84,87215,Central,Chris
2,31,19841,East,Erin
3,31,87300,East,Erin
4,28,19556,West,William
5,28,87957,West,William
6,27,19841,East,Erin
7,27,87300,East,Erin
8,24,91294,West,William
9,22,87572,South,Sam


# 11 . Group by region and find out which region is more profitable

In [165]:
pd.DataFrame(round(superstore.groupby('Region').agg({'Profit':np.sum}).nlargest(1,'Profit')))

,Profit
Region,
Central,519826.0


# 12 . Try to find out overall in which city we are giving more discount

In [174]:
superstore.groupby('State or Province').agg({'Discount':np.mean}).nlargest(1,'Discount')

,Discount
State or Province,
Wyoming,0.054762


# 13 . Give me a list of unique postal code

In [179]:
superstore['Postal Code'].unique()

array([60101, 98221, 91776, ..., 61832, 62521, 26554], dtype=int64)

# 14 . which customer segement is more profitalble find it out

In [185]:
round(superstore.groupby('Customer Segment').agg({'Profit':np.sum})).nlargest(1,'Profit')

,Profit
Customer Segment,
Corporate,505539.0


# 15 . try to find out the 10th most loss making product catagory

In [200]:
pd.DataFrame(superstore.groupby('Product Sub-Category').agg({'Profit':np.sum}).nsmallest(10,'Profit').iloc[9])

,Computer Peripherals
Profit,87917.842513


# 16 . Try to find out 10 top  product with highest margins 

In [222]:
round(superstore.groupby('Product Sub-Category').agg({'Product Base Margin':np.sum})).nlargest(10,'Product Base Margin')

,Product Base Margin
Product Sub-Category,
Telephones and Communication,577.0
Paper,517.0
Computer Peripherals,501.0
Office Furnishings,463.0
Storage & Organization,390.0
Binders and Binder Accessories,385.0
Pens & Art Supplies,385.0
Appliances,273.0
Tables,271.0
